# Imports

In [1]:
import os
import pathlib
import random
from math import sqrt
import ujson

import pandas as pd
import plotly.express as px

PATH = pathlib.Path.cwd() / "src"

# set current working directory to the path
os.chdir(PATH)
print(f"Current path: {PATH}")

Current path: /Users/eliebrosset/Documents/GitHub/cymoo/src


In [2]:
# update the PYTHONPATH
!export PYTHONPATH=$PYTHONPATH:{PATH}

In [3]:
from src.MOO import MOO
from src.optimizers.NSGA2 import NSGA2
from src.optimizers.NSRA import NSRA
from src.optimizers.NSWGE import NSWGE
from src.problems.multitask_routing.Network import Network
from src.problems.multitask_routing.Task import Task
from src.problems.Solution import Solution

# Defining the parameters

## Layers

In [4]:
# PROBLEM DEFINITION
paramsLayerOne = {
    "unit": {
        "tag": "DEVICE",
        "computingSpeed": lambda: random.randint(5, 10),
        "positionX": 0,
        "positionY": 0,
        "throughput": 0.5,
        "pollution": lambda: round(random.random(), 2),
        "cost": lambda: round(random.random(), 2),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 1,
        "flowRate": lambda: 1,
    },
    "numberNewUnits": 10,
}

paramsLayerTwo = {
    "unit": {
        "tag": "FOG",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(5, 10),
        "positionX": lambda x: x.positionX + random.randint(-2, 2),
        "positionY": lambda x: x.positionY + random.randint(-2, 2),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 2
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 20,
}

paramsLayerThree = {
    "unit": {
        "tag": "CLOUD",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(50, 100),
        "positionX": lambda x: x.positionX + random.randint(-5, 5),
        "positionY": lambda x: x.positionY + random.randint(-5, 5),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 3
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 10,
}

## Tasks

In [5]:
tasks = tuple(
    Task(random.randint(1, 10000), random.randint(1, 10000)) for _ in range(3)
)

## Network

In [6]:
optimUnits = {
    "processingTime": "Processing time (s)",
    "cost": "Cost (€)",
    "pollution": "Pollution (g)",
}
optimDirections = {
    "processingTime": "min",
    "cost": "min",
    "pollution": "min",
}
problem = Network(
    "DEVICE",
    tasks=tasks,
    optimDirections=optimDirections,
    minDepth=1,
    maxDepth=15,
    mutationRate=0.1,
    layers=[paramsLayerOne, paramsLayerTwo, paramsLayerThree],
)

## Optimization

In [7]:
nIterations = 10
nSolutions = 1000
seed = 10

# Optimization

## Training

In [8]:
moo = MOO(problem)

nsga2_paretos = moo.optimize(
    NSGA2,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
    seed=seed,
    ratioKept=0.5,
)
nsra_paretos = moo.optimize(
    NSRA,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
    seed=seed,
    ratioKept=0.5,
)
# nswge_paretos = moo.optimize(
#     NSWGE,
#     nSolutions,
#     nIterations,
#     imDir=PATH.parent / "imgs",
#     saveDir=PATH.parent / "paretos",
#     export=True,
#     seed=seed,
# )

Optimizing with NSGA2...


100%|██████████| 10/10 [00:05<00:00,  1.79it/s]


Displaying pareto solutions...
{'processingTime': 14089.0, 'cost': 11538.783940042282, 'pollution': 14880.118385274593}
{'processingTime': 14089.0, 'cost': 6066.1225, 'pollution': 3914.3811111111113}

Optimizing with NSRA...


100%|██████████| 10/10 [00:06<00:00,  1.65it/s]


Displaying pareto solutions...
{'processingTime': 34457.07974910394, 'cost': 42764.5250276211, 'pollution': 30088.116784918482}
{'processingTime': 35876.24358677449, 'cost': 26134.045334363636, 'pollution': 57266.42888995115}
{'processingTime': 34389.0238442279, 'cost': 59910.73752079059, 'pollution': 52506.250578084655}
{'processingTime': 34387.370980316235, 'cost': 64312.713223139435, 'pollution': 71146.00086238424}
{'processingTime': 34331.816326530614, 'cost': 87991.07222200725, 'pollution': 78720.81700013291}
{'processingTime': 34889.85339830925, 'cost': 27761.34415521082, 'pollution': 35143.41867659395}



## Evaluation

In [9]:
MOO.relative_efficiency(nsra_paretos, nsga2_paretos, Solution.optimDirections, verbose=True)

Relative efficiency: 0 % of <class 'src.optimizers.NSRA.NSRA'> solutions are undominated by <class 'src.optimizers.NSGA2.NSGA2'> solutions
Relative efficiency: 100 % of <class 'src.optimizers.NSGA2.NSGA2'> solutions are undominated by <class 'src.optimizers.NSRA.NSRA'> solutions


(0, 1)

# Results

In [10]:
def generate_violin_plot(df, title, yaxis_title="Value"):
    fig = px.violin(
        df,
        x="Epoch",
        y="Value",
        box=True,
        points=False,
        hover_data=df.columns,
        template="plotly_white",
        color_discrete_sequence=["#1f77b4"],
    )
    fig.update_layout(
        title=title,
        xaxis_title="Epoch",
        yaxis_title=yaxis_title,
        font=dict(
            family="Times New Roman",
            size=12,
        ),
    )
    fig.show()

In [11]:
# load pareto data from json
for file in os.listdir(PATH.parent / "paretos"):
    if file.endswith(".json"):
        # load file as dict
        print(file)
        with open(PATH.parent / "paretos" / file, "r") as f:
            data = ujson.load(f)
        for objective in optimDirections.keys():
            # unstack values
            df = (
                pd.DataFrame.from_dict(data[objective], orient="index")
                .stack()
                .reset_index()
                .rename(columns={"level_0": "Epoch", 0: "Value"})
                .drop(columns="level_1")
            )
            # generate plot
            generate_violin_plot(
                df,
                title=f"Violin plot of the pareto solutions for each epoch (10 epochs) for the {objective} objective",
                yaxis_title=optimUnits[objective],
            )


NSGA2.json


NSRA.json
